# Estudi Residus Municipals Catalunya 
## Data Cleaning

### Descripció
En aquest seguit de llibretes es recull l'estudi realitzat sobre el dataset de residus municipals a Catalunya publicat per la generalitat al portal de dades obertes. La intenció es investigar aquestes dades amb la fi de trobar conclusions interessants i possibles aplicacions.

En aquesta llibreta en concret es netegen les dades del dataset [Municipis Catalunya Geo](https://analisi.transparenciacatalunya.cat/Urbanisme-infraestructures/Municipis-Catalunya-Geo/9aju-tpwc) per a posteriorment ser enrriquides amb valors geografics i posteriorment realitzar un estudi sobre el territori que conforma Catalunya a nivell municipal.

### Autors
Joaquim Picó Mora, Marc Felip Pomes

In [4]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

### Lista de les primeres idees de cleening
- Molts nulls: Es pot deixar perque al final només volem visualitzar o es poden ficar a 0
    - Autocompostatge 
    - RAEE = Residus d'aparells electronics
    - Ferralla
    - Olis vegetals
    - Runes
    - Residus especials en petites quantitats
- Resta
    - Agrupar (Resta a Diposit + Resta a Incineració + Resta a tractament Mecànic Biològic) -> Resta (sense desglosar) 

In [3]:
spark = (SparkSession
 .builder
 .appName("WasteCleaning")
 .getOrCreate())

In [8]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [11]:
waste_file = "./datasets/Estad_stiques_de_residus_municipals.csv"
schema_df = "`Any` INT,\
            `Codi municipi` STRING,\
            `Municipi` STRING,\
            `Comarca` STRING,\
            `Població` FLOAT,\
            `Autocompostatge` FLOAT,\
            `Matèria orgànica` FLOAT,\
            `Poda i jardineria` FLOAT,\
            `Paper i cartró` FLOAT,\
            `Vidre` FLOAT,\
            `Envasos lleugers` FLOAT,\
            `Residus voluminosos + fusta` FLOAT,\
            `RAEE` FLOAT,\
            `Ferralla` FLOAT,\
            `Olis vegetals` FLOAT,\
            `Tèxtil` FLOAT,\
            `Runes` FLOAT,\
            `Residus Especials en petites quantitats (REPQ` FLOAT,\
            `Piles` FLOAT,\
            `Medicaments` FLOAT,\
            `Altres recollides selectives` FLOAT,\
            `R.S. / R.M. % total` FLOAT,\
            `Kg/hab/any recollida selectiva` FLOAT,\
            `Resta a Dipòsit` FLOAT,\
            `Resta a Incineració` FLOAT,\
            `Resta a Tractament Mecànic Biològic` FLOAT,\
            `Resta (sense desglossar)` FLOAT,\
            `Suma Fracció Resta` FLOAT,\
            `F.R. / R.M. %` FLOAT,\
            `Generació Residus Municipal Totals` FLOAT,\
            `Kg / hab / dia` FLOAT,\
            `Kg / hab / any` FLOAT"
df = spark.read.schema(schema_df).csv(waste_file)
df.show(n=5, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------
 Any                                           | null          
 Codi municipi                                 | Codi municipi 
 Municipi                                      | Municipi      
 Comarca                                       | Comarca       
 Població                                      | null          
 Autocompostatge                               | null          
 Matèria orgànica                              | null          
 Poda i jardineria                             | null          
 Paper i cartró                                | null          
 Vidre                                         | null          
 Envasos lleugers                              | null          
 Residus voluminosos + fusta                   | null          
 RAEE                                          | null          
 Ferralla                                      | null          
 Olis vegetals                          

In [16]:
df.summary()

summary,Any,Codi municipi,Municipi,Comarca,Població,Autocompostatge,Matèria orgànica,Poda i jardineria,Paper i cartró,Vidre,Envasos lleugers,Residus voluminosos + fusta,RAEE,Ferralla,Olis vegetals,Tèxtil,Runes,Residus Especials en petites quantitats (REPQ,Piles,Medicaments,Altres recollides selectives,R.S. / R.M. % total,Kg/hab/any recollida selectiva,Resta a Dipòsit,Resta a Incineració,Resta a Tractament Mecànic Biològic,Resta (sense desglossar),Suma Fracció Resta,F.R. / R.M. %,Generació Residus Municipal Totals,Kg / hab / dia,Kg / hab / any
count,18947,18948,18948,18948,18947,7583,18947,18947,18947,18947,18947,18947,9477,7583,7583,17053,8530,8530,18939,18939,18941,18947,18947,18947,15159,15159,15159,3788,18947,18947,18947,18947
mean,2009.5025597719955,211801.60728060442,null,null,1895.2873782033485,53.99854938678623,302.03608026951923,81.73821575453286,280.20880592826256,155.34687719001306,101.16649436068181,175.7145233874296,2776.366149625409,755.4793617301859,108.01450613213768,753.5698704040345,10429.165767878078,220.73950762016412,52.06980305190348,56.316700987380536,8380.519930309909,1306.4052432606702,30.232050465363066,161.78077163008007,97617.44290520482,33832.80836466786,78282.35945642853,171186.04567053853,2362.8534898082075,69.6571140595509,4055.2748115461295,1.52866522373916
stddev,5.76698664879997,125148.35428417475,null,null,51763.77802845096,153.66644831896883,3044.1352406414353,427.15700050310073,3659.6148645973544,1272.1979501894466,676.6865539597111,1239.9804421985236,19865.676319539198,2356.7832223221712,647.0445255900547,11059.452695505073,36868.55092180463,739.2123576764532,367.4630386695721,514.0797662252572,41737.07304731374,9736.829191720928,18.475453545301956,115.43738932153718,582131.1789086811,444922.2039185221,1018037.1097066668,1206595.4347514228,17477.475931780766,18.593340484707433,28010.944305577945,0.6567984435615438
min,2000,170010,Abella de la Conca,Alt Camp,0.0,0.0,0.0,0.0,0.0,0.0,-0.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2004,82397.0,null,null,4.069,0.0,0.0,0.0,7.7,8.63,3.65,0.15,36.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,33.28,15.47,80.33,2006.0,0.0,0.0,6193.0,92.99,58.63,158.72,1.15
50%,2010,171753.0,null,null,103.0,0.0,9.18,0.0,22.4,22.69,12.01,7.35,191.0,28.0,3.0,0.0,114.0,9.0,2.0,5.0,17.0,110.46,27.49,139.42,11315.0,0.0,0.0,22206.0,266.4,72.45,449.17,1.37
75%,2015,252075.0,null,null,346.0,48.0,118.62,16.8,97.27,79.27,50.35,55.44,1165.0,368.0,42.0,48.0,3428.0,118.0,16.0,22.0,637.0,544.34,41.29,215.72,43954.0,0.0,0.0,80627.0,1029.88,84.47,1909.16,1.69
max,2019,NA,Òrrius,Vallès Oriental,1636762.0,2764.0,128393.62,13457.29,176871.4,51232.54,25792.81,51485.3,793821.0,54487.0,23654.0,1103352.0,774636.0,22512.0,15071.0,27069.0,1416622.0,327502.88,100.0,2187.45,3.4422736E7,2.2374082E7,4.506766E7,6.667544E7,688440.44,100.0,902356.2,8.72


Ens interessa veure per quin motiu ho ha tants missing values a certs tipus de residus

In [48]:
print("("+"Any"+","+str(len(df.select(f.col("Any")).groupBy("Any").count().collect()))+")")

(Any,21)


In [51]:
columns = ["Autocompostatge", "RAEE", "Ferralla", "Olis vegetals", "Runes", "Residus Especials en petites quantitats (REPQ"]
for column in columns:
    print("("+column+","+str(len(df.select(f.col("Any"), f.col(column)).where(f.col(column) != None).groupBy("Any").count().collect()))+")")

(Autocompostatge,0)
(RAEE,0)
(Ferralla,0)
(Olis vegetals,0)
(Runes,0)
(Residus Especials en petites quantitats (REPQ,0)


In [49]:
print("("+"Municipi"+","+str(len(df.select(f.col("Municipi")).groupBy("Municipi").count().collect()))+")")

(Municipi,951)


In [50]:
columns = ["Autocompostatge", "RAEE", "Ferralla", "Olis vegetals", "Runes", "Residus Especials en petites quantitats (REPQ"]
for column in columns:
    print("("+column+","+str(len(df.select(f.col("Municipi"), f.col(column)).where(f.col(column) != None).groupBy("Municipi").count().collect()))+")")

(Autocompostatge,0)
(RAEE,0)
(Ferralla,0)
(Olis vegetals,0)
(Runes,0)
(Residus Especials en petites quantitats (REPQ,0)
